<a href="https://colab.research.google.com/github/Joan2022Laurente/fade/blob/main/notebooks/comfyui_colab_with_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 🛠️ ComfyUI Centralized Environment Setup
import os
from pathlib import Path

# --- CONFIGURACIÓN ---
HUB_NAME = "ComfyUI"  # El nombre de la carpeta compartida o acceso directo
WORKSPACE = f"/content/drive/MyDrive/{HUB_NAME}"
UPDATE_SYSTEM = True  # @param {type:"boolean"}

# 1. Montaje de Drive
if not os.path.exists('/content/drive'):
    print("📂 Montando Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')

# 2. Verificación de Rutas
if not os.path.exists(WORKSPACE):
    print(f"❌ ERROR: No se encontró la carpeta en {WORKSPACE}")
    print("Asegúrate de haber creado el 'Acceso directo' en esta cuenta secundaria.")
else:
    print(f"✅ Hub detectado en: {WORKSPACE}")
    %cd {WORKSPACE}

# 3. Instalación de Motor Ultra-Rápido (uv) e Inserción de Dependencias
print("⚡ Instalando dependencias con motor 'uv'...")
!pip install uv -q
# Instalamos 'av' junto con las librerías base para evitar errores de video
!uv pip install --system \
    accelerate einops transformers>=4.28.1 \
    safetensors>=0.4.2 aiohttp pyyaml Pillow \
    scipy tqdm psutil tokenizers>=0.13.3 \
    torchsde kornia>=0.7.1 spandrel soundfile \
    sentencepiece av GitPython -q

# 4. Actualización de ComfyUI y Manager
if UPDATE_SYSTEM:
    print("🔄 Sincronizando repositorio principal...")
    !git pull
    if os.path.exists("custom_nodes/ComfyUI-Manager"):
        %cd custom_nodes/ComfyUI-Manager
        !git pull
        %cd {WORKSPACE}

# 5. Restauración automática de dependencias de nodos
print("📦 Verificando dependencias de nodos instalados...")
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

📂 Montando Google Drive...
Mounted at /content/drive
✅ Hub detectado en: /content/drive/MyDrive/ComfyUI
/content/drive/MyDrive/ComfyUI
⚡ Instalando dependencias con motor 'uv'...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 122.8 MB/s eta 0:00:00
🔄 Sincronizando repositorio principal...
Already up to date.
/content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Already up to date.
/content/drive/MyDrive/ComfyUI
📦 Verificando dependencias de nodos instalados...

WARN: The `COMFYUI_PATH` environment variable is not set. Assuming 
`custom_nodes/ComfyUI-Manager/../../` as the ComfyUI path.
--------------------------------------------------------------------------------
--------------------
Restoring [1/1]: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Install: pip packages

## ComfyUI-Manager: EXECUTE => ['/usr/bin/python3', '-m', 'pip', 'install', 
'GitPython']

## ComfyUI-Manager: EXECUTE => ['/usr/bin/python3', '-m', 'uv', 'pip', 
'install', 'PyGithub']
Usi

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
# @title ⚡ Launch ComfyUI
import subprocess, threading, time, socket

PORT = 8188
cf_deb = f"{WORKSPACE}/cloudflared-linux-amd64.deb"

# Instalación persistente de Cloudflared
if not os.path.exists(cf_deb):
    print("📥 Descargando binario de Cloudflare al Hub...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O {cf_deb}

!dpkg -i {cf_deb} > /dev/null 2>&1

def cloudflare_tunnel(port):
    while True:
        time.sleep(1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            break
        sock.close()
    print("\n✨ Servidor listo. Generando enlace de acceso...\n")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print(f"🔗 URL DE ACCESO: {l[l.find('http'):]}")

threading.Thread(target=cloudflare_tunnel, daemon=True, args=(PORT,)).start()

# Ejecución limpia
!python main.py --dont-print-server --fast --preview-method auto

📥 Descargando binario de Cloudflare al Hub...
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-01-30 14:50:32.231
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/drive/MyDrive/ComfyUI
** ComfyUI Base Folder Path: /content/drive/MyDrive/ComfyUI
** User directory: /content/drive/MyDrive/ComfyUI/user
** ComfyUI-Manager config path: /content/drive/MyDrive/ComfyUI/user/__manager/config.ini
** Log path: /content/drive/MyDrive/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   7.0 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.0+cu126
Enabled fp16 accumulation.
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : native
Using async weight offloading with 2 streams
Enable